# <center>ANOM EYE AI - ANOMALY DETECTION IN SURVEILLANCE CAMERA USING MACHINE LEARNING</center>

# Anomaly Detection with Frame

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
#from twilio.rest import Client

categories_labels = {'Fighting': 0, 'Shoplifting': 1, 'Abuse': 2, 'Arrest': 3, 'Shooting': 4, 'Robbery': 5, 'Explosion': 6,'RoadAccidents': 7}
labels_categories = {v: k for k, v in categories_labels.items()}  # reverse dictionary for label lookup

# Load the trained model
model = load_model('/kaggle/input/my_model/keras/trained_model/1/CNN_LSTM model.h5')

def predict_image(image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Resize the image
    image = cv2.resize(image, (50, 50))

    # Reshape the image to 4D array for CNN and LSTM input
    image_cnn = image.reshape((1,) + image.shape + (1,))
    image_lstm = image.reshape((1,) + (-1, 1))

    # Use the model to predict the category of the image
    prediction = model.predict([image_cnn, image_lstm])
    
    # Find the category with the highest probability
    label = np.argmax(prediction)

    # Return the name of the category
    return labels_categories[label]

# Test the function
i2="/kaggle/input/roadaccidents/road_accident (701).jpg";
print(predict_image(i2));

def send_email(predict_image(i2)):
    MESSAGE = MIMEMultipart()
    MESSAGE['Subject'] = "Predicted Category"
    MESSAGE['From'] = FROM_EMAIL
    MESSAGE['To'] = TO_EMAIL

    text = f"""\
    <html>
    <body>
    <h3>⚠️ Security Alert</h3>
    <p>Our Anom Eye AI system has detected an ANOMALY: <span style="font-size: 20px; color: red;">{predict_image(i2)}</span></p>
    <p>Please review the alert promptly and take any necessary actions.</p>
    </body>
    </html>
    """
    
    html_part = MIMEText(text, 'html')
    MESSAGE.attach(html_part)

    # Attach image
    with open(image_path, 'rb') as img_file:
        img = MIMEImage(img_file.read())
        img.add_header('Content-Disposition', 'attachment', filename=os.path.basename(image_path))
        MESSAGE.attach(img)

    smtp = smtplib.SMTP(HOST, PORT)
    smtp.starttls()
    smtp.login(FROM_EMAIL, PASSWORD)

    smtp.sendmail(FROM_EMAIL, TO_EMAIL, MESSAGE.as_string())
    smtp.quit()


# ANOMALY DETECTION AND TESTING

In [ ]:
import cv2
import numpy as np
import os
from keras.models import load_model
from PIL import Image
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
import IPython.display as ipd
import time
!pip install twilio
from twilio.rest import Client

HOST = 'smtp-mail.outlook.com'
PORT = 587

FROM_EMAIL = "test_anomaly@outlook.com"
TO_EMAIL = "anomalydetection031@gmail.com"
PASSWORD = "******"

categories_labels = {'Fighting': 0, 'Shoplifting': 1, 'Abuse': 2, 'Arrest': 3, 'Shooting': 4, 'Robbery': 5, 'Explosion': 6}
labels_categories = {v: k for k, v in categories_labels.items()}  # reverse dictionary for label lookup
model = load_model('/kaggle/input/model/keras/trainedmodel/1/CNN_LSTM.h5')

def predict_image(image):
    # Resize the image
    image = cv2.resize(image, (50, 50))

    # Reshape the image to 4D array for CNN and LSTM input
    image_cnn = image.reshape((1,) + image.shape + (1,))
    image_lstm = image.reshape((1,) + (-1, 1))

    # Use the model to predict the category of the image
    prediction = model.predict([image_cnn, image_lstm])

    # Find the category with the highest probability
    label = np.argmax(prediction)

    # Return the name of the category
    return labels_categories[label]

def send_sms(predicted_category):
    account_sid = 'AC6304892eaaa621a46d7c5e73ad79c391'
    auth_token = 'e71f1d902972bde8833f11efd7b08583'
    # Twilio phone number and recipient's phone number
    twilio_number = '+12057547920'
    my_phone_number = '+91 1234567890'

    # Initialize Twilio client
    client = Client(account_sid, auth_token)

    # Send SMS
    message = client.messages.create(
        body=("Our Anom Eye AI system has detected an ANOMALY:"+predicted_category),
        from_=twilio_number,
        to=my_phone_number
    )
#     print("Message sent successfully! SID:", message.sid)


def send_email(predicted_category, image_path):
    MESSAGE = MIMEMultipart()
    MESSAGE['Subject'] = "Anomaly detected by ANOM EYE AI"
    MESSAGE['From'] = FROM_EMAIL
    MESSAGE['To'] = TO_EMAIL

    text = f"""\
    <html>
    <body>
    <h3>⚠️ Security Alert</h3>
    <p>Our Anom Eye AI system has detected an ANOMALY: <span style="font-size: 20px; color: red;">{predicted_category}</span></p>
    <p>Please review the alert promptly and take any necessary actions.</p>
    </body>
    </html>
    """
    
    html_part = MIMEText(text, 'html')
    MESSAGE.attach(html_part)

    # Attach image
    with open(image_path, 'rb') as img_file:
        img = MIMEImage(img_file.read())
        img.add_header('Content-Disposition', 'attachment', filename=os.path.basename(image_path))
        MESSAGE.attach(img)

    smtp = smtplib.SMTP(HOST, PORT)
    smtp.starttls()
    smtp.login(FROM_EMAIL, PASSWORD)

    smtp.sendmail(FROM_EMAIL, TO_EMAIL, MESSAGE.as_string())
    smtp.quit()

def extract_and_predict(video_path):
    # Open the video
    cap = cv2.VideoCapture(video_path)

    # Initialize variables
    frame_count = 0
    last_category = None

    # Loop through the frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Predict the category of the frame
        category = predict_image(frame_gray)

        # Print the category and frame count
        if last_category is None or category != last_category:
            print('Frame', frame_count, 'Category:', category)
            image_path = f'predicted_images/{category}_{frame_count}.png'
            cv2.imwrite(image_path, frame_gray)
            try:
                send_email(category, image_path)
                send_sms(category)
            except FileNotFoundError as e:
                print(f"Image file not found: {image_path}")
                print(e)
        last_category = category
        frame_count += 1
    cap.release()
# Test the function
video_path = "/kaggle/input/shoplifting/Shoplifting026_x264.mp4" 
os.makedirs('predicted_images', exist_ok=True)
# display_html = ipd.HTML(f'<video width="320" height="240" autoplay controls><source src="{video_path}" type="video/mp4"></video>')
# display(display_html)
extract_and_predict(video_path)